In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
import os
import gdown

print('Python', sys.version)
dataset_path = os.environ.get('RCMD_DATASET_PATH', '../dataset')
print('Dataset path', dataset_path)
for i in [pd, np]:
    print(i.__name__, i.__version__)

pd.options.display.float_format = "{:.6g}".format

Python 3.12.11 (main, Dec 12 2025, 14:54:08) [GCC 13.3.0]
Dataset path ../dataset
pandas 2.3.3
numpy 2.3.5


# MovieLens

[MovieLens](https://grouplens.org/datasets/movielens/)

Minnesota 대학의 GroupLens 연구소에서 제작한 데이터입니다. 

미네소타 대학에서 운영하는 [MovieLens](https://movielens.org/) 사이트에서 1995년 부터 2023년 까지 수집한 평점 데이터셋입니다.

본 수업에서는 MovieLens 100K 를 통해 여러가지 추천시스템에 있어 기본적인 요소를 살펴보고,

규모가 큰 MovieLens 32M 을 통해 규모가 보다 규모가 큰 추천 시스템을 다룹니다.

## 데이터 다운로드

본 과정에서 다루는 데이터셋은 아래 구글 드라이브에 있습니다

[Google Drive](https://drive.google.com/drive/folders/1B2MWhhEjf1HChP85n9mp8Bp-UvqdvLLA)

아래는 Google Drive에 있는 데이터셋을 로컬의 dataset 경로에 다운로드 받게하는 코드입니다.

In [2]:
# 데이터를 다운로드 받습니다.
if not os.path.exists(dataset_path):
    os.mkdir(dataset_path)
dataset_files =  [
    ('movies_100k.csv', '1SX84B13wp4OFP8W1lPjJNK3NZewEKGg_'),
    ('ratings_100k.csv', '1ey4ilNGEtcHiVLzQZ5j8dQsv98Nv46pC'),
    ('users_100k.csv', '1ekjE0fPq5YF06X0vTdtnX57hE0PsLK-R'),
    ('movies.csv', '1N-WAyRgx69S8bEnK3jE29NDXR0NgA5rg'),
    ('ratings.parquet', '1CBUt0L4GjOFaJ5llCq24MI8cma-R8NZF'),
    ('tmdb_movie_emb_kr.pkl', '1KUJDU81o2S4KLO4ohKkyqpCg-60pw2gM'),
    ('tmdb_movie_emb.pkl', '1oS1MC0SOS7wFTg_hdbrR0uvGXoc6sR3G'),
    ('tmdb_movie_info_kr.pkl', '1YEXzzE7YdoqsY6wIKSCdhpHUfb_Eynrv'),
    ('tmdb_movie_info.pkl', '1hbKv1E7mXnK6yKtoGNsU7pQviKtihHH5'),
]
for f, i in dataset_files:
    fname = os.path.join(dataset_path, f)
    if not os.path.exists(fname):
        o = gdown.download("https://drive.google.com/uc?id={}".format(i), quiet=False)
        os.rename(o, fname)

In [3]:
os.listdir(dataset_path), len(os.listdir(dataset_path))

(['movies.csv',
  'movies_100k.csv',
  'ratings.parquet',
  'ratings_100k.csv',
  'tmdb_movie_emb.pkl',
  'tmdb_movie_emb_kr.pkl',
  'tmdb_movie_info.pkl',
  'tmdb_movie_info_kr.pkl',
  'users_100k.csv'],
 9)

## 1. MovieLens 100K


MovieLens에서 가장 규모가 작은 데이터셋입니다. 

단순하지만, 추천 시스템을 공부하기 위해 여러 가지 시도를 해볼수 있는 데이터셋입니다.

|파일|내용|크기|특징|
|--|---|---|---|
|movies_100k.csv|제목/개봉일/장르|1,682건|복수의 장르를 담고 있습니다.|
|users_100k.csv|성별/나이/직업/zipcode|943건||
|ratings.csv|사용자별 영화 평점|100,000|평점은 0.0~5.0까지 1점 단위입니다.<br/>유저 당 20건이상 평점이 있습니다.|

In [4]:
df_movies_100k = pd.read_csv(os.path.join(dataset_path, 'movies_100k.csv'), index_col='movie id')
df_movies_100k

,movie title,release date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,Mat' i syn (1997),06-Feb-1998,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1679,B. Monkey (1998),06-Feb-1998,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1680,Sliding Doors (1998),01-Jan-1998,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
df_users_100k = pd.read_csv(os.path.join(dataset_path, 'users_100k.csv'), index_col='user id')
df_users_100k

,age,gender,occupation,zip code
user id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213
...,...,...,...,...
939,26,F,student,33319
940,32,M,administrator,02215
941,20,M,student,97229


In [6]:
!head ../dataset/ratings_100k.csv

user id,item id,rating,timestamp
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596
298,474,4,884182806
115,265,2,881171488
253,465,5,891628467
305,451,3,886324817


In [7]:
df_ratings_100k = pd.read_csv(os.path.join(dataset_path, 'ratings_100k.csv'))
df_ratings_100k['date'] = df_ratings_100k.pop('timestamp').pipe(lambda x: pd.to_datetime(x, unit='s')) 
df_ratings_100k

,user id,item id,rating,date
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16
...,...,...,...,...
99995,880,476,3,1997-11-22 05:10:44
99996,716,204,5,1997-11-17 19:39:03
99997,276,1090,1,1997-09-20 22:49:55
99998,13,225,2,1997-12-17 22:52:36


## 2. MovieLens 32m

MovieLens에서 1997년 1월에서 2023년 10월까지 수집한 데이터입니다.

이를 기반으로 본 수업에서 필요한 데이터만 발췌했습니다. 

* MovieLens 32m에서 발췌한 데이터

|파일|내용|크기|특징|
|--|---|---|---|
|movies.csv|영화 기본정보|87,585건|출시 연도는 제목에 포함되어 있습니다.|
|ratings.csv|사용자별 영화 평점|32,000,204건|평점은 0.0~5.0까지 0.5점 단위입니다.<br/>유저 당 20건이상 평점이 있습니다.|

* 수집한 데이터

|파일|내용|크기|특징|
|----|----|----|----|
|tmdb_movie_info.pkl|tmdb에서 수집한 영화 상세정보|86,577|영문|
|tmdb_movie_info_kr.pkl|tmdb에서 수집한 영화 상세정보|86,576|국문|
|tmdb_movie_emb.pkl|영화 줄거리|86,577|openAI embedding API를 통해 추출한 텍스트 수치화 정보. 영문|
|tmdb_movie_emb_kr.pkl|영화 줄거리|86,576|openAI embedding API를 통해 추출한 텍스트 수치화 정보. 국문|

*tmdb: 영화정보제공 서비스

*[openAI embeddings](https://platform.openai.com/docs/guides/embeddings/what-are-embeddings): text-embedding-ada-002 모델을 통해 텍스트를 1536개의 수치 데이터로 변환

### rating.parquet

In [8]:
# Rating 데이터를 불러와 봅니다.
df_ratings = pd.read_parquet(os.path.join(dataset_path, 'ratings.parquet'))
# df_ratings DataFrame에서 timestamp Series를 꺼낸 다음(pop)
# 정수형 Timestamp를 일자형식으로 바꿉니다.
df_ratings['date'] = df_ratings.pop('timestamp').pipe(lambda x: pd.to_datetime(x, unit='s')) 
df_ratings

,userId,movieId,rating,date
0,1,17,4,1999-12-03 19:24:37
1,1,25,1,1999-12-03 19:43:48
2,1,29,2,1999-11-22 00:36:16
3,1,30,5,1999-12-03 19:24:37
4,1,32,5,1999-11-22 00:00:58
...,...,...,...,...
32000199,200948,79702,4.5,2011-01-07 15:03:09
32000200,200948,79796,1,2010-10-16 08:04:52
32000201,200948,80350,0.5,2011-01-07 15:04:31
32000202,200948,80463,3.5,2012-10-16 21:43:20


### Movie.csv

In [9]:
# movie.csv 시작 부분을 출력해봅니다.
movie_filename = os.path.join(dataset_path, 'movies.csv')
!head $movie_filename

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action


In [10]:
# 영화 기본 정보를 가져옵니다. 
# movieId는 고유 정보이므로 인덱스로 사용합니다.
df_movie = pd.read_csv(movie_filename, index_col='movieId')
df_movie

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
292731,The Monroy Affaire (2022),Drama
292737,Shelter in Solitude (2023),Comedy|Drama
292753,Orca (2023),Drama


### 영화 상세 정보

영화 상세정보는 [tMdb](https://www.themoviedb.org/)에서 제공하는 API를 사용하여 영문/한글 두 가지 언어로 수집했습니다.

내용은 Python dictionary 형태로 되어 있습니다. Key: movieId, Value: 영화 상세 정보

효율적으로 저장하기 위해 Python pickle을 사용하여 저장되어 있습니다.

In [11]:
# 영화 상세 정보(국문)를 불러 옵니다.
with open(os.path.join(dataset_path, 'tmdb_movie_info_kr.pkl'), 'rb') as f:
    dic_movie_info_kr = pickle.load(f)
dic_movie_info_kr[1] # movieId = 1(Toy Story)의 상세 정보를 봅니다.

{'adult': False,
 'backdrop_path': '/3Rfvhy1Nl6sSGJwyjb0QiZzZYlB.jpg',
 'belongs_to_collection': {'id': 10194,
  'name': '토이 스토리 시리즈',
  'poster_path': '/rki5qLuwb0xnnE9seehxO9TlLhW.jpg',
  'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'},
 'budget': 30000000,
 'genres': [{'id': 16, 'name': '애니메이션'},
  {'id': 12, 'name': '모험'},
  {'id': 10751, 'name': '가족'},
  {'id': 35, 'name': '코미디'}],
 'homepage': '',
 'id': 862,
 'imdb_id': 'tt0114709',
 'original_language': 'en',
 'original_title': 'Toy Story',
 'overview': '카우보이 인형 우디는 꼬마 주인인 앤디의 가장 사랑받는 장난감이다. 그러나 어느날 버즈라는 새로운 장난감이 등장한다.  버즈는 최신형 장난감으로 레이저 빔 등의 첨단장비를 갖추고 있으나, 버즈는 자신이 장난감임을 인식하지 못하고 자신이 우주에서 온 전사이며 자신이 갖춘 장비로 하늘을 날 수 있다고 믿고 있다. 버즈의 허상을 상처받지 않고 인식시켜 주려는 우디와 친구들. 그러나 뜻밖의 사고가 일어난다. 옆집 개에게 버즈가 납치당하고 이런 버즈를 구하기 위해 우디와 친구들은 구조대를 조직해 버즈를 구하러 가는데...',
 'popularity': 102.848,
 'poster_path': '/5ELwzkC7QY9vug20AvRFOXBzLbG.jpg',
 'production_companies': [{'id': 3,
   'logo_path': '/1TjvGVDMYsj6JBxOAkUHpPEwLf7.png',
   'name': 'Pixar',
 

In [12]:
# 영화 상세 정보(영문)를 불러 옵니다.
with open(os.path.join(dataset_path, 'tmdb_movie_info.pkl'), 'rb') as f:
    dic_movie_info_en = pickle.load(f)
dic_movie_info_en[1]  # movieId = 1(Toy Story)의 상세 정보를 봅니다.

{'adult': False,
 'backdrop_path': '/3Rfvhy1Nl6sSGJwyjb0QiZzZYlB.jpg',
 'belongs_to_collection': {'id': 10194,
  'name': 'Toy Story Collection',
  'poster_path': '/rki5qLuwb0xnnE9seehxO9TlLhW.jpg',
  'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'},
 'budget': 30000000,
 'genres': [{'id': 16, 'name': 'Animation'},
  {'id': 12, 'name': 'Adventure'},
  {'id': 10751, 'name': 'Family'},
  {'id': 35, 'name': 'Comedy'}],
 'homepage': 'http://toystory.disney.com/toy-story',
 'id': 862,
 'imdb_id': 'tt0114709',
 'original_language': 'en',
 'original_title': 'Toy Story',
 'overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
 'popularity': 102.848,
 'poster_path': '/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg',
 'production_companies': [{'id'

In [13]:
len(dic_movie_info_kr), len(dic_movie_info_en)

(86576, 86577)

## 영화 줄거리 Embedding

**Embedding은 객체나 개념을 수치화한 정보를 뜻합니다.**

영화 줄거리 Embedding은 tMdb를 통해 수집된 영화 정보 중에 제목(title)과 줄거리(overview)를 

openAI에서 제공하는 text embedding 서비스를 통해 수치화한 데이터입니다. - [OpenAI - Embeddings](https://platform.openai.com/docs/guides/embeddings)

text-embedding-ada-002 모델을 사용하여 영화 줄거리 하나당  얻은 1536개의 수치 정보로 변환했습니다.

수집한 Toy Story의 영화 정보에서 overview

```
카우보이 인형 우디는 꼬마 주인인 앤디의 가장 사랑받는 장난감이다. 그러나 어느날 버즈라는 새로운 장난감이 등장한다.  버즈는 최신형 장난감으로 레이저 빔 등의 첨단장비를 갖추고 있으나, 버즈는 자신이 장난감임을 인식하지 못하고 자신이 우주에서 온 전사이며 자신이 갖춘 장비로 하늘을 날 수 있다고 믿고 있다. 버즈의 허상을 상처받지 않고 인식시켜 주려는 우디와 친구들. 그러나 뜻밖의 사고가 일어난다. 옆집 개에게 버즈가 납치당하고 이런 버즈를 구하기 위해 우디와 친구들은 구조대를 조직해 버즈를 구하러 가는데...
```
를 openAI Embedding 서비스에 전달하여 변환한 정보입니다.


In [14]:
# 영화 줄거리에 대한 openAI embedding을 통한 수치화 정보를 불러 옵니다.
with open(os.path.join(dataset_path, 'tmdb_movie_emb_kr.pkl'), 'rb') as f:
    dic_movie_emb = pickle.load(f)
dic_movie_emb[1] # movieId = 1(Toy Story)의 줄거리 수치화 정보입니다.

array([-0.00791671, -0.03155841,  0.00588331, ..., -0.00125478,
       -0.00891308, -0.01079059], shape=(1536,))

In [15]:
dic_movie_emb[1], dic_movie_emb[1].shape

(array([-0.00791671, -0.03155841,  0.00588331, ..., -0.00125478,
        -0.00891308, -0.01079059], shape=(1536,)),
 (1536,))

In [16]:
len(dic_movie_emb)

86576

영문 버젼의 overview

Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences

를 openAI Embedding 서비스로 뽑아낸 정보입니다.

In [17]:
# 영화 줄거리에 대한 openAI embedding을 통한 수치화 정보를 불러 옵니다.
with open(os.path.join(dataset_path, 'tmdb_movie_emb.pkl'), 'rb') as f:
    dic_movie_emb = pickle.load(f)
dic_movie_emb[1] # movieId = 1(Toy Story)의 줄거리 수치화 정보입니다.

array([-0.0131833 , -0.04171353, -0.01378756, ...,  0.01245558,
       -0.0124166 , -0.02548294], shape=(1536,))

In [18]:
len(dic_movie_emb)

86577